## Felles import for alle kodeblokker

In [1]:
from monthly_statistics import MonthlyStatistics
from analysis import DataAnalyzer 

analyzer = DataAnalyzer("../../data/processed")
stats = MonthlyStatistics("../../data/processed")

## Eksempl koder på bruk av analyse funksjonene

### Viser statistikk ved en eller alle måneder

In [2]:
# Regn ut statistikk for én måned i Oslo
resultat = stats.compute_single_month(
    "2024-03", "mean(air_temperature P1D)", "oslo", time_offset="PT0H"
)
print(resultat)

# Hent alle måneder for Tromsø
df = stats.compute_all_months(
    "range(air_temperature P1D)", "tromso", time_offset="PT18H"
)
df.head()

{'mean': 2.6967741935483875, 'median': 2.7, 'std': 2.2953647360696494}


,year_month,mean,median,std
0,2000-01,4.232258,4.00,1.700076
1,2000-02,4.382759,4.40,1.797234
2,2000-03,4.448387,4.20,1.928708
3,2000-04,5.206667,4.85,1.972734
4,2000-05,5.525806,5.50,1.808677


### Finner antall eksterm verdier

In [3]:
df_outliers = analyzer.finn_outliers_per_maaned(
    by="oslo",
    element_id="sum(precipitation_amount P1D)",
    time_offset="PT18H",
    vis_tomme_maaneder=True
)
df_outliers.head()

,year_month,outliers_removed,antall_totalt,andel_outliers_%
0,2000-01,7,31,22.6
1,2000-02,5,29,17.2
2,2000-03,6,31,19.4
3,2000-04,0,30,0.0
4,2000-05,3,31,9.7


### Finner statistisk verdier med og uten ekstrem verdier

In [4]:
df_stat = analyzer.statistikk_med_og_uten_outliers(
    by="tromso",
    element_id="mean(air_temperature P1D)",
    time_offset="PT0H",
    statistikk="mean"
)
df_stat.head()

,year_month,mean_with_outliers,mean_without_outliers,outliers_removed,elementId
0,2000-01,-2.400,-2.400,0,mean(air_temperature P1D)
1,2000-02,-3.297,-3.297,0,mean(air_temperature P1D)
2,2000-03,-3.113,-3.113,0,mean(air_temperature P1D)
3,2000-04,-0.810,-0.810,0,mean(air_temperature P1D)
4,2000-05,4.965,4.965,0,mean(air_temperature P1D)


### Henter og viser manglende verdier i en DF

In [5]:
df_missing = analyzer.les_manglende_verdier("../../data/missing/missing_in_both.csv")
df_missing.head()

FileNotFoundError: [Errno 2] No such file or directory: '../../data/missing/missing_in_both.csv'

### Viser prosentvis endring fra en periode til neste

In [6]:
df_endring = analyzer.prosentvis_endring(
    by="oslo",
    element_id="sum(precipitation_amount P1D)",
    time_offset="PT18H",
    statistikk="mean",
    frekvens="YE",  # "D" for dag, "ME" for måned, "YE" for år
    start="2020-01",
    end="2024-01"
)
df_endring.head()
 

,periode,verdi,prosent_endring
0,2021-12-31 00:00:00+00:00,1.908219,-35.302620
1,2022-12-31 00:00:00+00:00,1.922466,0.746590
2,2023-12-31 00:00:00+00:00,2.802740,45.788799
3,2024-12-31 00:00:00+00:00,10.400000,271.065494
